In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import  torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter
random.seed(1024)
flatten = lambda l: [item    for sublist in l     for item in sublist]

FloatTensor = torch.FloatTensor
LongTensor = torch.LongTensor
ByteTensor = torch.ByteTensor



In [2]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch
    
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]))

In [3]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:500]
corpus = [[word.lower() for word in sent] for sent in corpus]

word_count = Counter(flatten(corpus))

MIN_COUNT = 3
exclude = []
#少于出现3次的排除到exclude数组
for w, c in word_count.items():
    if c < MIN_COUNT:
        exclude.append(w)
        
vocab = list(set(flatten(corpus)) - set(exclude))
word2index = {}
for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
        
index2word = {v:k for k, v in word2index.items()}


WINDOW_SIZE = 5
windows =  flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1)) for c in corpus])

train_data = []

for window in windows:
    for i in range(WINDOW_SIZE * 2 + 1):
        if window[i] in exclude or window[WINDOW_SIZE] in exclude: 
            continue # min_count
        if i == WINDOW_SIZE or window[i] == '<DUMMY>': 
            continue
        train_data.append((window[WINDOW_SIZE], window[i]))

X_p = []
y_p = []

for tr in train_data:
    X_p.append(prepare_word(tr[0], word2index).view(1, -1))
    y_p.append(prepare_word(tr[1], word2index).view(1, -1))
    
train_data = list(zip(X_p, y_p))

print(len(train_data))
print(train_data[:3])

50242
[(tensor([[ 57]]), tensor([[ 411]])), (tensor([[ 57]]), tensor([[ 0]])), (tensor([[ 57]]), tensor([[ 93]]))]


In [4]:
Z = 0.001

word_count = Counter(flatten(corpus))
num_total_words = sum([c for w, c in word_count.items() if w not in exclude])

unigram_table = []
#unigram  一元分布
for vo in vocab:
    unigram_table.extend( [vo] * int(   (  (word_count[vo]/num_total_words )**0.75 )/Z)   )

print(len(vocab), len(unigram_table))
# print(unigram_table)

478 3500


In [5]:
def negative_sampling(targets, unigram_table, k):
    batch_size = targets.size(0)
    neg_samples = []
    for i in range(batch_size):
        nsample = []
        target_index = targets[i].data.tolist()[0]
        while len(nsample) < k: # num of sampling
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).view(1, -1))
    
    return torch.cat(neg_samples)



class SkipgramNegSampling(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(SkipgramNegSampling, self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim) # center embedding
        self.embedding_u = nn.Embedding(vocab_size, projection_dim) # out embedding
        self.logsigmoid = nn.LogSigmoid()
                
        initrange = (2.0 / (vocab_size + projection_dim))**0.5 # Xavier init
        self.embedding_v.weight.data.uniform_(-initrange, initrange) # init
        self.embedding_u.weight.data.uniform_(-0.0, 0.0) # init
        
    def forward(self, center_words, target_words, negative_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D            V_c   
        target_embeds = self.embedding_u(target_words) # B x 1 x D             U_o
        
        neg_embeds = -self.embedding_u(negative_words) # B x K x D            U_w
        
        positive_score = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1     V_c*U_o = Bx1 
        negative_score = torch.sum(neg_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2), 1).view(negs.size(0), -1) # BxK -> Bx1
        
        loss = self.logsigmoid(positive_score) + self.logsigmoid(negative_score)
        #目标，整体的loss和最小
        return -torch.mean(loss)
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds

In [6]:
EMBEDDING_SIZE = 30 
BATCH_SIZE = 256
EPOCH = 100
NEG = 10 # Num of Negative Sampling  ===K

losses = []
model = SkipgramNegSampling(len(word2index), EMBEDDING_SIZE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCH):
    for i,batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs) # B x 1
        targets = torch.cat(targets) # B x 1
        negs = negative_sampling(targets, unigram_table, NEG)
        model.zero_grad()

        loss = model(inputs, targets, negs)
        
        loss.backward()
        optimizer.step()
    
        losses.append(loss.data.tolist())
    if epoch % 10 == 0:
        print("Epoch : %d, mean_loss : %.02f" % (epoch, np.mean(losses)))
        losses = []

Epoch : 0, mean_loss : 1.06


Epoch : 10, mean_loss : 0.86


Epoch : 20, mean_loss : 0.79


Epoch : 30, mean_loss : 0.74


Epoch : 40, mean_loss : 0.71


Epoch : 50, mean_loss : 0.69


Epoch : 60, mean_loss : 0.67


Epoch : 70, mean_loss : 0.65


Epoch : 80, mean_loss : 0.64


Epoch : 90, mean_loss : 0.63


In [8]:
def word_similarity(target, vocab):
    target_V = model.prediction(prepare_word(target, word2index))
    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target: 
            continue
        vector = model.prediction(prepare_word(list(vocab)[i], word2index)) 
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0]
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:50]

test = random.choice(list(vocab))
print(test)
word_similarity(test, vocab)

less


[['forty', 0.6766588687896729],
 ['stone', 0.6119797229766846],
 ['other', 0.598942756652832],
 ['than', 0.5670243501663208],
 ['ago', 0.5607035756111145],
 ['has', 0.5506868958473206],
 ['.--', 0.5298324823379517],
 ['young', 0.521355390548706],
 ['did', 0.5118181109428406],
 ['true', 0.5105212330818176],
 ['very', 0.4924517571926117],
 ['bones', 0.4904303252696991],
 ['pacific', 0.4853748083114624],
 ['after', 0.47465184330940247],
 ['no', 0.4651760160923004],
 ['thinks', 0.4535908102989197],
 ['craft', 0.45033183693885803],
 ['ocean', 0.44850897789001465],
 ['four', 0.4480888545513153],
 ['having', 0.4445367455482483],
 ['jaw', 0.43572139739990234],
 ['further', 0.43286415934562683],
 ['account', 0.4305078685283661],
 ['ten', 0.42733919620513916],
 ['london', 0.42642566561698914],
 ['battle', 0.4253599941730499],
 ['cape', 0.4241832196712494],
 ['will', 0.4207867681980133],
 ['about', 0.4201004207134247],
 ['else', 0.41658487915992737],
 ['some', 0.41565465927124023],
 ['because', 0